# Needed imports

In [21]:
import nltk
import os

In [22]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\abdal\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# Data Preprocessing

In [23]:
dataset_path = "./data"
data_text = ""
files = os.listdir(dataset_path)
for file_index in range(1000):
    filename = files[file_index]
    with open(os.path.join(dataset_path, filename), 'r', encoding='utf-8') as file:
        data_text += file.read()


sentences = data_text.split('\n')
    
# Remove leading and trailing spaces
sentences = [s.strip() for s in sentences]

# Drop Empty Sentences
sentences = [s for s in sentences if len(s) > 0]

# Empty List to hold Tokenized Sentences
tokens = []
count = 0

# Iterate through sentences
for sentence in sentences:
		
	# Convert to lowercase
	sentence = sentence.lower()
	
	# Tokenize the sentence
	tokenized_sentence = nltk.word_tokenize(sentence)
	
	# Append to list
	tokens.append(tokenized_sentence)
	count += len(tokenized_sentence)

print("Sample of size: ", count, "is used")

Sample of size:  838472 is used


In [24]:
# Calculating frequencies using a dictionary
trigrams = {}
for tokenized_sentence in tokens:
    tokenized_sentence = ["<s>", "<s>"] + tokenized_sentence
    for i in range(len(tokenized_sentence) - 2):
        trigram = (tokenized_sentence[i], tokenized_sentence[i+1], tokenized_sentence[i+2])
        trigrams[trigram] = trigrams.get(trigram, 0) + 1

# Sort the dictionary based on the frequency count of each trigram
trigrams = {k: v for k, v in sorted(trigrams.items(), key=lambda item: item[1], reverse=True)}

# Define a function to suggest the next word based on the last two words of the input sentence
def get_suggestions(sentence):
    words = ["<s>", "<s>"] + nltk.word_tokenize(sentence.lower())
    if len(words) == 2:
        candidates = []
    else:
        last_two_words = tuple(words[-2:])
        candidates = [k[2] for k in trigrams.keys() if k[:2] == last_two_words]
    return candidates

# GUI

In [25]:
import tkinter as tk
root = tk.Tk()
root.title("Auto-filling model using trigram")

# create label and entry widgets
tk.Label(root, text="Enter text:").grid(row=0, column=0, padx=5, pady=5)
entry = tk.Entry(root, width=50)
entry.grid(row=0, column=1, padx=5, pady=5)

# create suggestions list widget
tk.Label(root, text="Suggestions:").grid(row=1, column=0, padx=5, pady=5)
listbox = tk.Listbox(root, height=5, width=50)
listbox.grid(row=1, column=1, padx=5, pady=5)

# define function to update suggestions list
def update_suggestions(event):
    if event.keysym == "Return":
        sentence = entry.get()
        suggestions = get_suggestions(sentence)[:5]
        listbox.delete(0, tk.END)
        if(not suggestions):
            listbox.insert(tk.END, "--")
        for suggestion in suggestions:
            listbox.insert(tk.END, suggestion)

# bind entry widget to update_suggestions function
entry.bind("<KeyRelease>", update_suggestions)

root.mainloop()